# Распознавание букв Backpropagation
Чуть изменённая задача из книги Роберта Каллана 2001 гг. "Основные концепции нейронных сетей". стр. 63 "Классификация символов"

Реализация в Numpy (успех зависит от нач иниц)

Классификация символов A, B, C, D. Чёрно-белые изображения букв 8 на 8, по 5 картинок на букву (обучение - по 4, 2 тестовые)

Сойдёт использование 2-х сигмоид и выходные векторы как 00 для A, 01 для B и т.д. 

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from PIL import Image
path = "D:/pseudo_data/"
a = []; b = []; c = []; d = []
n_imgperl = 6
for i in range(n_imgperl):
    a.append(Image.open(path + f"a{i}.png"))
    b.append(Image.open(path + f"b{i}.png"))
    c.append(Image.open(path + f"c{i}.png"))
    d.append(Image.open(path + f"d{i}.png"))

In [2]:
def PixelArray(img):
    img = np.array(img)
    oz = np.zeros((8, 8))
    for i in range(img.shape[0]):
        oz[i] = [x[0] for x in img[i]]
    oz[oz == 0] = 1
    oz[oz > 1] = 0 # 255 - это белый, белый пусть будет 0
    return oz

In [3]:
letter0 = np.array([PixelArray(a[0]), PixelArray(b[0]), PixelArray(c[0]), PixelArray(d[0])])
letter1 = np.array([PixelArray(a[1]), PixelArray(b[1]), PixelArray(c[1]), PixelArray(d[1])])
letter2 = np.array([PixelArray(a[2]), PixelArray(b[2]), PixelArray(c[2]), PixelArray(d[2])])
letter3 = np.array([PixelArray(a[3]), PixelArray(b[3]), PixelArray(c[3]), PixelArray(d[3])])
letter4 = np.array([PixelArray(a[4]), PixelArray(b[4]), PixelArray(c[4]), PixelArray(d[4])])
letter5 = np.array([PixelArray(a[5]), PixelArray(b[5]), PixelArray(c[5]), PixelArray(d[5])])

In [4]:
answers = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
#answers = np.zeros((4, 4))
#for i in range(4):
#    answers[i, i] = 1
#X = np.concatenate((letter1, letter2, letter3, letter4, letter5))
X = np.concatenate((letter0, letter1, letter2, letter3)) ; Y = np.concatenate((answers, answers, answers, answers))
#Y = np.concatenate((answers, answers, answers, answers, answers))

In [5]:
def net(x, w):
    return np.dot(x, w)
def sigmoid(wx):
    return 1 / (1 + np.exp(-wx))
def sigderiv(sig_neuron):
    return sig_neuron * (1 - sig_neuron)

In [6]:
def forward_pass(x, w_hid, w_out):
    neurons = sigmoid(net(x, w_hid))
    output = sigmoid(net(neurons, w_out))
    return neurons, output
def predict(x, w_hid, w_out):
    x = np.append(x.reshape(1, -1), [[-1]])
    neurons = sigmoid(net(x, w_hid))
    output = sigmoid(net(neurons, w_out))
    return output

In [7]:
def Learning(epsilon, X, Y, num_hid, lmd):
    inp_size = np.size(X[0]) + 1
    out = Y[0].size
    w_hid = np.random.uniform(-2, 2, (inp_size, num_hid))
    #w_hid = np.zeros((inp_size, hid))
    w_out = np.random.uniform(-2, 2, (num_hid, out))
# обучение
    error = np.array([float('inf'), float('inf')])
    mean_error = float('inf')
    epoch = 0
    while (mean_error > epsilon):
    #for epoch in range(10000):
        mean_error = 0
        for i in range(X.shape[0]):
            x = np.append(X[i], [[-1]])
            x = x.reshape(1, -1) #shape: 1, 64 (65 т.к. фиктивный признак 1)
            y = Y[i]
            neurons, output = forward_pass(x, w_hid, w_out)
            error = y - output
            #sigderiv 1, 2
            delta = error * sigderiv(output) #1, 2 локальный градиент выходного нейрона
            #print(f'delta {sigderiv(output).shape}')
            delta2 = sigderiv(neurons) * np.dot(delta, np.transpose(w_out)) #лок град 2
            w_out += lmd * np.dot(np.transpose(neurons), delta)
            w_hid += lmd * np.dot(np.transpose(x), delta2) #64 8
            #print(output, y)
            mean_error += np.sum(abs(error))
        mean_error /= X.shape[0] * error.size
        epoch += 1
        if epoch % 1000 == 0:
            print(f'\t{epoch} {mean_error}')
    return w_hid, w_out, epoch

In [8]:
hidden_weights, output_weights, number_of_epochs = Learning(1e-3, X, Y, 3, 9e-1)

	1000 0.012885818807012341
	2000 0.00865989721866226
	3000 0.006861858859187533
	4000 0.005833865708007756
	5000 0.005154524223557005
	6000 0.004665080593819669
	7000 0.00429175929483834
	8000 0.003995289575233629
	9000 0.0037526828903003758
	10000 0.0035495031907657117
	11000 0.003376182463949157
	12000 0.0032261020976902797
	13000 0.0030945219584016166
	14000 0.0029779482300770713
	15000 0.0028737426481348794
	16000 0.002779871433841862
	17000 0.0026947385962845134
	18000 0.002617072074149469
	19000 0.002545844029122456
	20000 0.002480213827086789
	21000 0.002419486460604814
	22000 0.0023630817089172872
	23000 0.002310510909318392
	24000 0.0022613592178760216
	25000 0.002215271891364209
	26000 0.00217194355700731
	27000 0.0021311097311242522
	28000 0.002092540050696684
	29000 0.0020560328239309163
	30000 0.0020214106067533264
	31000 0.001988516584762747
	32000 0.001957211593039397
	33000 0.0019273716451676966
	34000 0.0018988858718419156
	35000 0.0018716547912424482
	36000 0.00184558

In [9]:
print(f"На обучение было потрачено {number_of_epochs} эпох.")
for i in range(letter1.shape[0]):
    #print(f'\t{[round(i) for i in predict(letter3[i], hidden_weights, output_weights)]} {answers[i]}')
    print(f'\t{predict(letter5[i], hidden_weights, output_weights)} {answers[i]}')

На обучение было потрачено 125711 эпох.
	[0.0015731  0.00141296] [0 0]
	[0.00139243 0.99967104] [0 1]
	[9.97842194e-01 3.92324120e-04] [1 0]
	[0.99842752 0.99821392] [1 1]


In [10]:
print(f"На обучение было потрачено {number_of_epochs} эпох.")
for i in range(letter1.shape[0]):
    #print(f'\t{[round(i) for i in predict(letter3[i], hidden_weights, output_weights)]} {answers[i]}')
    print(f'\t{predict(letter4[i], hidden_weights, output_weights)} {answers[i]}')

На обучение было потрачено 125711 эпох.
	[0.00160036 0.00158216] [0 0]
	[0.001241   0.99959304] [0 1]
	[0.98414519 0.00153035] [1 0]
	[0.99851579 0.99834027] [1 1]
